# 16. Movielens 영화 Session Based Recommendation

## 데이터 준비

In [1]:
import pandas as pd
from pathlib import Path
import os
import numpy as np

import warnings

warnings.filterwarnings("ignore")

<IPython.core.display.Javascript object>

In [2]:
data_path = Path(os.getenv("HOME") + "/aiffel/yoochoose/data/")
train_path = data_path / "ratings.dat"


def load_data(data_path: Path, nrows=None):
    data = pd.read_csv(
        data_path,
        sep="::",
        header=None,
        usecols=[0, 1, 2, 3],
        dtype={0: np.int32, 1: np.int32, 2: np.int32},
        nrows=nrows,
    )
    data.columns = ["UserId", "ItemId", "Rating", "Time"]
    return data


data = load_data(train_path, None)
data.sort_values(["UserId", "Time"], inplace=True)

data_orig = data.copy()

<IPython.core.display.Javascript object>

In [3]:
data["UserId"].nunique(), data["ItemId"].nunique()

(6040, 3706)

<IPython.core.display.Javascript object>

사용자는 6040명, 영화는 3706개의 항목이 존재합니다.

데이터셋에 세션 항목이 존재하지 않습니다. `UserId`와 `Time` 정보를 사용하여 새로운 `SessionId`를 부여하겠습니다.  
동일 유저가 특정한 기간동안 기록한 활동을 하나의 세션으로 취급합니다.  
우선 1시간을 기준으로 세션화를 진행합니다.

In [4]:
unit = 3600 * 1
data["TimeDisc"] = data["Time"].floordiv(unit) * unit
data

,UserId,ItemId,Rating,Time,TimeDisc
31,1,3186,4,978300019,978300000
22,1,1270,5,978300055,978300000
27,1,1721,4,978300055,978300000
37,1,1022,5,978300055,978300000
24,1,2340,3,978300103,978300000
...,...,...,...,...,...
1000019,6040,2917,4,997454429,997452000
999988,6040,1921,4,997454464,997452000
1000172,6040,1784,3,997454464,997452000
1000167,6040,161,3,997454486,997452000


<IPython.core.display.Javascript object>

In [5]:
grp = data.groupby(["UserId", "TimeDisc"]).count().reset_index()
grp["SessionId"] = grp.index
grp.drop(["ItemId", "Rating", "Time"], axis=1, inplace=True)
grp

,UserId,TimeDisc,SessionId
0,1,978300000,0
1,1,978822000,1
2,2,978296400,2
3,2,978300000,3
4,3,978296400,4
...,...,...,...
31964,6040,964825200,31964
31965,6040,964828800,31965
31966,6040,984193200,31966
31967,6040,997452000,31967


<IPython.core.display.Javascript object>

In [6]:
data_merge = pd.merge(data, grp)
data_merge

,UserId,ItemId,Rating,Time,TimeDisc,SessionId
0,1,3186,4,978300019,978300000,0
1,1,1270,5,978300055,978300000,0
2,1,1721,4,978300055,978300000,0
3,1,1022,5,978300055,978300000,0
4,1,2340,3,978300103,978300000,0
...,...,...,...,...,...,...
1000204,6040,2917,4,997454429,997452000,31967
1000205,6040,1921,4,997454464,997452000,31967
1000206,6040,1784,3,997454464,997452000,31967
1000207,6040,161,3,997454486,997452000,31967


<IPython.core.display.Javascript object>

In [7]:
data = data_merge.drop(["TimeDisc"], axis=1)
data

,UserId,ItemId,Rating,Time,SessionId
0,1,3186,4,978300019,0
1,1,1270,5,978300055,0
2,1,1721,4,978300055,0
3,1,1022,5,978300055,0
4,1,2340,3,978300103,0
...,...,...,...,...,...
1000204,6040,2917,4,997454429,31967
1000205,6040,1921,4,997454464,31967
1000206,6040,1784,3,997454464,31967
1000207,6040,161,3,997454486,31967


<IPython.core.display.Javascript object>

In [8]:
session_length = data.groupby("SessionId").size()
session_length

SessionId
0         40
1         13
2        109
3         20
4         51
        ... 
31964     13
31965      1
31966      1
31967     21
31968      1
Length: 31969, dtype: int64

<IPython.core.display.Javascript object>

세션에 대한 통계를 확인합니다.

In [9]:
session_length.median(), session_length.mean()

(8.0, 31.286840376614844)

<IPython.core.display.Javascript object>

In [10]:
session_length.min(), session_length.max()

(1, 711)

<IPython.core.display.Javascript object>

In [11]:
length_count = session_length.groupby(session_length).size()
length_percent_cumsum = length_count.cumsum() / length_count.sum()
length_percent_cumsum_999 = length_percent_cumsum[length_percent_cumsum < 0.999]

length_percent_cumsum_999

1      0.237480
2      0.329663
3      0.378742
4      0.413056
5      0.445400
         ...   
389    0.998874
391    0.998905
392    0.998936
395    0.998968
401    0.998999
Length: 359, dtype: float64

<IPython.core.display.Javascript object>

길이가 3 이하인 세션이 전체 세션의 약 38% 입니다.

1시간이 아닌 0.5시간(30분) 또는 2시간을 기준으로 세션화를 진행하면 결과가 어떻게 변화하는지 확인합니다.

In [12]:
data = data_orig.copy()
unit = 3600 * 0.5
data["TimeDisc"] = data["Time"].floordiv(unit) * unit
grp = data.groupby(["UserId", "TimeDisc"]).count().reset_index()
grp["SessionId"] = grp.index
grp.drop(["ItemId", "Rating", "Time"], axis=1, inplace=True)
data_merge = pd.merge(data, grp)
data = data_merge.drop(["TimeDisc"], axis=1)
session_length = data.groupby("SessionId").size()
length_count = session_length.groupby(session_length).size()
length_percent_cumsum = length_count.cumsum() / length_count.sum()
length_percent_cumsum_999 = length_percent_cumsum[length_percent_cumsum < 0.999]

length_percent_cumsum_999

1      0.218633
2      0.305704
3      0.354733
4      0.392162
5      0.425971
         ...   
241    0.998725
243    0.998776
244    0.998827
245    0.998929
246    0.998980
Length: 242, dtype: float64

<IPython.core.display.Javascript object>

In [13]:
data = data_orig.copy()
unit = 3600 * 2
data["TimeDisc"] = data["Time"].floordiv(unit) * unit
grp = data.groupby(["UserId", "TimeDisc"]).count().reset_index()
grp["SessionId"] = grp.index
grp.drop(["ItemId", "Rating", "Time"], axis=1, inplace=True)
data_merge = pd.merge(data, grp)
data = data_merge.drop(["TimeDisc"], axis=1)
session_length = data.groupby("SessionId").size()
length_count = session_length.groupby(session_length).size()
length_percent_cumsum = length_count.cumsum() / length_count.sum()
length_percent_cumsum_999 = length_percent_cumsum[length_percent_cumsum < 0.999]

length_percent_cumsum_999

1      0.252293
2      0.348250
3      0.397209
4      0.430361
5      0.459551
         ...   
525    0.998787
528    0.998822
532    0.998858
536    0.998894
537    0.998965
Length: 443, dtype: float64

<IPython.core.display.Javascript object>

0.5시간, 1시간, 2시간으로 세션화를 진행한 결과 세션의 길이가 3 이하인 경우는 40% 이하입니다.

In [14]:
data = data_orig.copy()
unit = 3600 * 1
data["TimeDisc"] = data["Time"].floordiv(unit) * unit
grp = data.groupby(["UserId", "TimeDisc"]).count().reset_index()
grp["SessionId"] = grp.index
grp.drop(["ItemId", "Rating", "Time"], axis=1, inplace=True)
data_merge = pd.merge(data, grp)
data = data_merge.drop(["TimeDisc"], axis=1)

<IPython.core.display.Javascript object>

추천하기에 적절하지 않은 항목들을 제외합니다.
- `Rating`을 1 또는 2 를 주었다는 것은 영화에 대한 흥미가 떨어진다는 것으로 추천을 하지 않는 것이 좋기에 데이터셋에서 제외합니다.
- 평가가 10번 이하로 이루어진 영화는 인기가 없는 영화일 확률이 높기에 제외합니다
- 세션의 길이가 3 미만인 데이터는 제외합니다.

In [15]:
data.drop(data[data.Rating < 3].index, inplace=True)
data

,UserId,ItemId,Rating,Time,SessionId
0,1,3186,4,978300019,0
1,1,1270,5,978300055,0
2,1,1721,4,978300055,0
3,1,1022,5,978300055,0
4,1,2340,3,978300103,0
...,...,...,...,...,...
1000204,6040,2917,4,997454429,28021
1000205,6040,1921,4,997454464,28021
1000206,6040,1784,3,997454464,28021
1000207,6040,161,3,997454486,28021


<IPython.core.display.Javascript object>

In [16]:
def cleanse_recursive(data: pd.DataFrame, shortest, least_click) -> pd.DataFrame:
    while True:
        before_len = len(data)
        data = cleanse_short_session(data, shortest)
        data = cleanse_unpopular_item(data, least_click)
        after_len = len(data)
        if before_len == after_len:
            break
    return data


def cleanse_short_session(data: pd.DataFrame, shortest):
    session_len = data.groupby("SessionId").size()
    session_use = session_len[session_len >= shortest].index
    data = data[data["SessionId"].isin(session_use)]
    return data


def cleanse_unpopular_item(data: pd.DataFrame, least_click):
    item_popular = data.groupby("ItemId").size()
    item_use = item_popular[item_popular >= least_click].index
    data = data[data["ItemId"].isin(item_use)]
    return data

<IPython.core.display.Javascript object>

In [17]:
data = cleanse_recursive(data, shortest=3, least_click=10)
data

,UserId,ItemId,Rating,Time,SessionId
0,1,3186,4,978300019,0
1,1,1270,5,978300055,0
2,1,1721,4,978300055,0
3,1,1022,5,978300055,0
4,1,2340,3,978300103,0
...,...,...,...,...,...
1000203,6040,232,5,997454398,28021
1000204,6040,2917,4,997454429,28021
1000205,6040,1921,4,997454464,28021
1000206,6040,1784,3,997454464,28021


<IPython.core.display.Javascript object>

각 세션이 기록된 시간을 확인합니다.

In [18]:
oldest, latest = data["Time"].min(), data["Time"].max()
oldest = pd.to_datetime(oldest, unit="s")
latest = pd.to_datetime(latest, unit="s")
oldest, latest

(Timestamp('2000-04-25 23:05:32'), Timestamp('2003-02-28 17:49:50'))

<IPython.core.display.Javascript object>

In [19]:
(latest - oldest).days

1038

<IPython.core.display.Javascript object>

데이터셋은 1,038일 동안의 정보를 담고 있습니다. validation을 위한 데이터에 200일, test를 위한 데이터에 200일 동안의 데이터를 사용합니다.

In [20]:
import datetime as dt


def split_by_date(data: pd.DataFrame, n_days: int):
    final_time = data["Time"].max()
    session_last_time = data.groupby("SessionId")["Time"].max()
    session_in_train = session_last_time[
        session_last_time < final_time - n_days * 86400
    ].index
    session_in_test = session_last_time[
        session_last_time >= final_time - n_days * 86400
    ].index

    before_date = data[data["SessionId"].isin(session_in_train)]
    after_date = data[data["SessionId"].isin(session_in_test)]
    after_date = after_date[after_date["ItemId"].isin(before_date["ItemId"])]
    return before_date, after_date

<IPython.core.display.Javascript object>

In [21]:
tr, test = split_by_date(data, n_days=200)
tr, val = split_by_date(tr, n_days=200)

<IPython.core.display.Javascript object>

In [22]:
def stats_info(data: pd.DataFrame, status: str):
    print(
        f"* {status} Set Stats Info\n"
        f"\t Events: {len(data)}\n"
        f'\t Sessions: {data["SessionId"].nunique()}\n'
        f'\t Items: {data["ItemId"].nunique()}\n'
        f'\t First Time : {pd.to_datetime(data["Time"].min(), unit="s")}\n'
        f'\t Last Time : {pd.to_datetime(data["Time"].max(), unit="s")}\n'
    )

<IPython.core.display.Javascript object>

In [23]:
stats_info(tr, "train")
stats_info(val, "valid")
stats_info(test, "test")

* train Set Stats Info
	 Events: 805419
	 Sessions: 16309
	 Items: 3100
	 First Time : 2000-04-25 23:05:32
	 Last Time : 2002-01-24 03:46:12

* valid Set Stats Info
	 Events: 9853
	 Sessions: 698
	 Items: 2284
	 First Time : 2002-01-24 20:01:17
	 Last Time : 2002-08-12 13:53:29

* test Set Stats Info
	 Events: 7188
	 Sessions: 462
	 Items: 1986
	 First Time : 2002-08-13 09:05:17
	 Last Time : 2003-02-28 17:49:50



<IPython.core.display.Javascript object>

In [24]:
# train set에 없는 아이템이 val, test기간에 생길 수 있으므로 train data를 기준으로 인덱싱합니다.
id2idx = {item_id: index for index, item_id in enumerate(tr["ItemId"].unique())}


def indexing(df, id2idx):
    df["item_idx"] = df["ItemId"].map(
        lambda x: id2idx.get(x, -1)
    )  # id2idx에 없는 아이템은 모르는 값(-1) 처리 해줍니다.
    return df


tr = indexing(tr, id2idx)
val = indexing(val, id2idx)
test = indexing(test, id2idx)

<IPython.core.display.Javascript object>

## 데이터셋 구성

In [25]:
class SessionDataset:
    """Credit to yhs-968/pyGRU4REC."""

    def __init__(self, data):
        self.df = data
        self.click_offsets = self.get_click_offsets()
        self.session_idx = np.arange(
            self.df["SessionId"].nunique()
        )  # indexing to SessionId

    def get_click_offsets(self):
        """
        Return the indexes of the first click of each session IDs,
        """
        offsets = np.zeros(self.df["SessionId"].nunique() + 1, dtype=np.int32)
        offsets[1:] = self.df.groupby("SessionId").size().cumsum()
        return offsets

<IPython.core.display.Javascript object>

In [26]:
tr_dataset = SessionDataset(tr)
tr_dataset.df.head(10)

,UserId,ItemId,Rating,Time,SessionId,item_idx
0,1,3186,4,978300019,0,0
1,1,1270,5,978300055,0,1
2,1,1721,4,978300055,0,2
3,1,1022,5,978300055,0,3
4,1,2340,3,978300103,0,4
5,1,1836,5,978300172,0,5
6,1,3408,4,978300275,0,6
7,1,2804,5,978300719,0,7
8,1,1207,4,978300719,0,8
9,1,1193,5,978300760,0,9


<IPython.core.display.Javascript object>

In [27]:
tr_dataset.click_offsets

array([     0,     40,     53, ..., 805386, 805398, 805419], dtype=int32)

<IPython.core.display.Javascript object>

In [28]:
tr_dataset.session_idx

array([    0,     1,     2, ..., 16306, 16307, 16308])

<IPython.core.display.Javascript object>

In [29]:
class SessionDataLoader:
    """Credit to yhs-968/pyGRU4REC."""

    def __init__(self, dataset: SessionDataset, batch_size=64):
        self.dataset = dataset
        self.batch_size = batch_size

    def __iter__(self):
        """Returns the iterator for producing session-parallel training mini-batches.
        Yields:
            input (B,):  Item indices that will be encoded as one-hot vectors later.
            target (B,): a Variable that stores the target item indices
            masks: Numpy array indicating the positions of the sessions to be terminated
        """

        (
            start,
            end,
            mask,
            last_session,
            finished,
        ) = self.initialize()  # initialize 메소드에서 확인해주세요.
        """
        start : Index Where Session Start
        end : Index Where Session End
        mask : indicator for the sessions to be terminated
        """

        while not finished:
            min_len = (end - start).min() - 1  # Shortest Length Among Sessions
            for i in range(min_len):
                # Build inputs & targets
                inp = self.dataset.df["item_idx"].values[start + i]
                target = self.dataset.df["item_idx"].values[start + i + 1]
                yield inp, target, mask

            start, end, mask, last_session, finished = self.update_status(
                start, end, min_len, last_session, finished
            )

    def initialize(self):
        first_iters = np.arange(self.batch_size)  # 첫 배치에 사용할 세션 Index를 가져옵니다.
        last_session = self.batch_size - 1  # 마지막으로 다루고 있는 세션 Index를 저장해둡니다.
        start = self.dataset.click_offsets[
            self.dataset.session_idx[first_iters]
        ]  # data 상에서 session이 시작된 위치를 가져옵니다.
        end = self.dataset.click_offsets[
            self.dataset.session_idx[first_iters] + 1
        ]  # session이 끝난 위치 바로 다음 위치를 가져옵니다.
        mask = np.array([])  # session의 모든 아이템을 다 돌은 경우 mask에 추가해줄 것입니다.
        finished = False  # data를 전부 돌았는지 기록하기 위한 변수입니다.
        return start, end, mask, last_session, finished

    def update_status(
        self,
        start: np.ndarray,
        end: np.ndarray,
        min_len: int,
        last_session: int,
        finished: bool,
    ):
        # 다음 배치 데이터를 생성하기 위해 상태를 update합니다.

        start += min_len  # __iter__에서 min_len 만큼 for문을 돌았으므로 start를 min_len 만큼 더해줍니다.
        mask = np.arange(self.batch_size)[(end - start) == 1]
        # end는 다음 세션이 시작되는 위치인데 start와 한 칸 차이난다는 것은 session이 끝났다는 뜻입니다. mask에 기록해줍니다.

        for i, idx in enumerate(mask, start=1):  # mask에 추가된 세션 개수만큼 새로운 세션을 돌것입니다.
            new_session = last_session + i
            if (
                new_session > self.dataset.session_idx[-1]
            ):  # 만약 새로운 세션이 마지막 세션 index보다 크다면 모든 학습데이터를 돈 것입니다.
                finished = True
                break
            # update the next starting/ending point
            start[idx] = self.dataset.click_offsets[
                self.dataset.session_idx[new_session]
            ]  # 종료된 세션 대신 새로운 세션의 시작점을 기록합니다.
            end[idx] = self.dataset.click_offsets[
                self.dataset.session_idx[new_session] + 1
            ]

        last_session += len(mask)  # 마지막 세션의 위치를 기록해둡니다.
        return start, end, mask, last_session, finished

<IPython.core.display.Javascript object>

In [30]:
tr_data_loader = SessionDataLoader(tr_dataset)
tr_dataset.df.head(15)

,UserId,ItemId,Rating,Time,SessionId,item_idx
0,1,3186,4,978300019,0,0
1,1,1270,5,978300055,0,1
2,1,1721,4,978300055,0,2
3,1,1022,5,978300055,0,3
4,1,2340,3,978300103,0,4
5,1,1836,5,978300172,0,5
6,1,3408,4,978300275,0,6
7,1,2804,5,978300719,0,7
8,1,1207,4,978300719,0,8
9,1,1193,5,978300760,0,9


<IPython.core.display.Javascript object>

성능을 평가하기 위한 metric 함수를 정의합니다.

In [31]:
def mrr_k(pred, truth: int, k: int):
    indexing = np.where(pred[:k] == truth)[0]
    if len(indexing) > 0:
        return 1 / (indexing[0] + 1)
    else:
        return 0


def recall_k(pred, truth: int, k: int) -> int:
    answer = truth in pred[:k]
    return int(answer)

<IPython.core.display.Javascript object>

## 모델 학습 및 평가

In [32]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, GRU
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm

<IPython.core.display.Javascript object>

In [33]:
def create_model(args):
    inputs = Input(batch_shape=(args.batch_size, 1, args.num_items))
    gru, _ = GRU(args.hsz, stateful=True, return_state=True, name="GRU")(inputs)
    dropout = Dropout(args.drop_rate)(gru)
    predictions = Dense(args.num_items, activation="softmax")(dropout)
    model = Model(
        inputs=inputs, outputs=[predictions], name=f"m_b{args.batch_size}_h{args.hsz}"
    )
    model.compile(
        loss=categorical_crossentropy, optimizer=Adam(args.lr), metrics=["accuracy"]
    )
    model.summary()
    return model

<IPython.core.display.Javascript object>

In [34]:
BATCH_SZ = 128
H_SZ = 64
EPOCH = 10

<IPython.core.display.Javascript object>

In [35]:
class Args:
    def __init__(self, tr, val, test, batch_size, hsz, drop_rate, lr, epochs, k):
        self.tr = tr
        self.val = val
        self.test = test
        self.num_items = tr["ItemId"].nunique()
        self.num_sessions = tr["SessionId"].nunique()
        self.batch_size = batch_size
        self.hsz = hsz
        self.drop_rate = drop_rate
        self.lr = lr
        self.epochs = epochs
        self.k = k


args = Args(
    tr,
    val,
    test,
    batch_size=BATCH_SZ,
    hsz=H_SZ,
    drop_rate=0.1,
    lr=0.001,
    epochs=EPOCH,
    k=20,
)

<IPython.core.display.Javascript object>

In [36]:
model = create_model(args)

Model: "m_b128_h64"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(128, 1, 3100)]          0         
_________________________________________________________________
GRU (GRU)                    [(128, 64), (128, 64)]    607872    
_________________________________________________________________
dropout (Dropout)            (128, 64)                 0         
_________________________________________________________________
dense (Dense)                (128, 3100)               201500    
Total params: 809,372
Trainable params: 809,372
Non-trainable params: 0
_________________________________________________________________


<IPython.core.display.Javascript object>

In [37]:
# train 셋으로 학습하면서 valid 셋으로 검증합니다.
def train_model(model, args, verbose=False):
    train_dataset = SessionDataset(args.tr)
    train_loader = SessionDataLoader(train_dataset, batch_size=args.batch_size)

    for epoch in range(1, args.epochs + 1):
        total_step = len(args.tr) - args.tr["SessionId"].nunique()
        tr_loader = tqdm(
            train_loader,
            total=total_step // args.batch_size,
            desc="Train",
            mininterval=1,
        )
        for feat, target, mask in tr_loader:
            reset_hidden_states(
                model, mask
            )  # 종료된 session은 hidden_state를 초기화합니다. 아래 메서드에서 확인해주세요.

            input_ohe = to_categorical(feat, num_classes=args.num_items)
            input_ohe = np.expand_dims(input_ohe, axis=1)
            target_ohe = to_categorical(target, num_classes=args.num_items)

            result = model.train_on_batch(input_ohe, target_ohe)
            tr_loader.set_postfix(train_loss=result[0], accuracy=result[1])

        val_recall, val_mrr = get_metrics(
            args.val, model, args, args.k
        )  # valid set에 대해 검증합니다.
        if verbose == 1 and epoch and epoch % 5 == 0:
            print(f"\t - Recall@{args.k} epoch {epoch}: {val_recall:3f}")
            print(f"\t - MRR@{args.k}    epoch {epoch}: {val_mrr:3f}")
        if verbose == 2 and epoch:
            print(f"\t - Recall@{args.k} epoch {epoch}: {val_recall:3f}")
            print(f"\t - MRR@{args.k}    epoch {epoch}: {val_mrr:3f}")


def reset_hidden_states(model, mask):
    gru_layer = model.get_layer(name="GRU")  # model에서 gru layer를 가져옵니다.
    hidden_states = gru_layer.states[0].numpy()  # gru_layer의 parameter를 가져옵니다.
    for elt in mask:  # mask된 인덱스 즉, 종료된 세션의 인덱스를 돌면서
        hidden_states[elt, :] = 0  # parameter를 초기화 합니다.
    gru_layer.reset_states(states=hidden_states)


def get_metrics(data, model, args, k: int):  # valid셋과 test셋을 평가하는 코드입니다.
    # train과 거의 같지만 mrr, recall을 구하는 라인이 있습니다.
    dataset = SessionDataset(data)
    loader = SessionDataLoader(dataset, batch_size=args.batch_size)
    recall_list, mrr_list = [], []

    total_step = len(data) - data["SessionId"].nunique()
    for inputs, label, mask in tqdm(
        loader, total=total_step // args.batch_size, desc="Evaluation", mininterval=1
    ):
        reset_hidden_states(model, mask)
        input_ohe = to_categorical(inputs, num_classes=args.num_items)
        input_ohe = np.expand_dims(input_ohe, axis=1)

        pred = model.predict(input_ohe, batch_size=args.batch_size)
        pred_arg = tf.argsort(
            pred, direction="DESCENDING"
        )  # softmax 값이 큰 순서대로 sorting 합니다.

        length = len(inputs)
        recall_list.extend([recall_k(pred_arg[i], label[i], k) for i in range(length)])
        mrr_list.extend([mrr_k(pred_arg[i], label[i], k) for i in range(length)])

    recall, mrr = np.mean(recall_list), np.mean(mrr_list)
    return recall, mrr

<IPython.core.display.Javascript object>

In [38]:
train_model(model, args, verbose=2)

Evaluation:  68%|██████▊   | 48/71 [00:26<00:12,  1.83it/s]


	 - Recall@20 epoch 1: 0.115723
	 - MRR@20    epoch 1: 0.029443


Evaluation:  68%|██████▊   | 48/71 [00:25<00:12,  1.86it/s]


	 - Recall@20 epoch 2: 0.143555
	 - MRR@20    epoch 2: 0.037504


Evaluation:  68%|██████▊   | 48/71 [00:25<00:12,  1.89it/s]


...

Evaluation:  68%|██████▊   | 48/71 [00:25<00:12,  1.90it/s]


	 - Recall@20 epoch 9: 0.162923
	 - MRR@20    epoch 9: 0.048023


Evaluation:  68%|██████▊   | 48/71 [00:25<00:12,  1.90it/s]

	 - Recall@20 epoch 10: 0.166829
	 - MRR@20    epoch 10: 0.048603


<IPython.core.display.Javascript object>

In [39]:
def test_model(model, args, test):
    test_recall, test_mrr = get_metrics(test, model, args, 20)
    print()
    print("====Evaluation====")
    print(f"\t - Recall@{args.k}: {test_recall:3f}")
    print(f"\t - MRR@{args.k}: {test_mrr:3f}\n")


test_model(model, args, test)

Evaluation:  56%|█████▌    | 29/52 [00:15<00:11,  1.92it/s]


====Evaluation====
	 - Recall@20: 0.170259
	 - MRR@20: 0.054571



<IPython.core.display.Javascript object>

## 모델 탐색

### 세션 정의 변경

In [40]:
def get_data(d_orig):
    data = d_orig.copy()
    data["TimeDisc"] = data["Time"].floordiv(unit) * unit
    grp = data.groupby(["UserId", "TimeDisc"]).count().reset_index()
    grp["SessionId"] = grp.index
    grp.drop(["ItemId", "Rating", "Time"], axis=1, inplace=True)
    data_merge = pd.merge(data, grp)
    data = data_merge.drop(["TimeDisc"], axis=1)
    data.drop(data[data.Rating < 3].index, inplace=True)
    data = cleanse_recursive(data, shortest=3, least_click=10)
    return data

<IPython.core.display.Javascript object>

In [41]:
unit = 0.5
data = get_data(data_orig)

tr, test = split_by_date(data, n_days=200)
tr, val = split_by_date(tr, n_days=200)

id2idx = {item_id: index for index, item_id in enumerate(tr["ItemId"].unique())}

tr = indexing(tr, id2idx)
val = indexing(val, id2idx)
test = indexing(test, id2idx)

args = Args(
    tr,
    val,
    test,
    batch_size=BATCH_SZ,
    hsz=H_SZ,
    drop_rate=0.1,
    lr=0.001,
    epochs=EPOCH,
    k=20,
)
model = create_model(args)
train_model(model, args, verbose=1)
test_model(model, args, test)

Model: "m_b128_h64"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(128, 1, 2667)]          0         
_________________________________________________________________
GRU (GRU)                    [(128, 64), (128, 64)]    524736    
_________________________________________________________________
dropout_1 (Dropout)          (128, 64)                 0         
_________________________________________________________________
dense_1 (Dense)              (128, 2667)               173355    
Total params: 698,091
Trainable params: 698,091
Non-trainable params: 0
_________________________________________________________________


Evaluation:  94%|█████████▍| 16/17 [00:07<00:00,  2.12it/s]


	 - Recall@20 epoch 5: 0.312988
	 - MRR@20    epoch 5: 0.107193


Evaluation:  94%|█████████▍| 16/17 [00:07<00:00,  2.17it/s]


	 - Recall@20 epoch 10: 0.326172
	 - MRR@20    epoch 10: 0.112007


Evaluation:  89%|████████▉ | 8/9 [00:03<00:00,  2.06it/s]


====Evaluation====
	 - Recall@20: 0.263672
	 - MRR@20: 0.099476



<IPython.core.display.Javascript object>

In [42]:
unit = 2
data = get_data(data_orig)

tr, test = split_by_date(data, n_days=200)
tr, val = split_by_date(tr, n_days=200)

id2idx = {item_id: index for index, item_id in enumerate(tr["ItemId"].unique())}

tr = indexing(tr, id2idx)
val = indexing(val, id2idx)
test = indexing(test, id2idx)

args = Args(
    tr,
    val,
    test,
    batch_size=BATCH_SZ,
    hsz=H_SZ,
    drop_rate=0.1,
    lr=0.001,
    epochs=EPOCH,
    k=20,
)
model = create_model(args)
train_model(model, args, verbose=1)
test_model(model, args, test)

Model: "m_b128_h64"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(128, 1, 2673)]          0         
_________________________________________________________________
GRU (GRU)                    [(128, 64), (128, 64)]    525888    
_________________________________________________________________
dropout_2 (Dropout)          (128, 64)                 0         
_________________________________________________________________
dense_2 (Dense)              (128, 2673)               173745    
Total params: 699,633
Trainable params: 699,633
Non-trainable params: 0
_________________________________________________________________


Evaluation:  89%|████████▉ | 16/18 [00:07<00:00,  2.12it/s]


	 - Recall@20 epoch 5: 0.305664
	 - MRR@20    epoch 5: 0.102995


Evaluation:  89%|████████▉ | 16/18 [00:07<00:00,  2.13it/s]


	 - Recall@20 epoch 10: 0.314453
	 - MRR@20    epoch 10: 0.110491


Evaluation:  90%|█████████ | 9/10 [00:04<00:00,  2.04it/s]


====Evaluation====
	 - Recall@20: 0.265625
	 - MRR@20: 0.091663



<IPython.core.display.Javascript object>

세션을 정의하는 시간의 범위가 1시간인 경우와 비교하여 30분, 2시간인 경우 성능이 향상되었습니다. Recall의 성능 차이는 크지 않으나, 30분인 경우의 MRR이 조금 더 증가하여 1시간 보다 더 작은 세션 단위에 대한 추가 실험을 진행합니다.  
세션을 정의하는 시간을 15분으로 설정하여 실험을 진행합니다.

In [43]:
unit = 0.25
data = get_data(data_orig)

tr, test = split_by_date(data, n_days=200)
tr, val = split_by_date(tr, n_days=200)

id2idx = {item_id: index for index, item_id in enumerate(tr["ItemId"].unique())}

tr = indexing(tr, id2idx)
val = indexing(val, id2idx)
test = indexing(test, id2idx)

args = Args(
    tr,
    val,
    test,
    batch_size=BATCH_SZ,
    hsz=H_SZ,
    drop_rate=0.1,
    lr=0.001,
    epochs=EPOCH,
    k=20,
)
model = create_model(args)
train_model(model, args, verbose=1)
test_model(model, args, test)

Model: "m_b128_h64"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(128, 1, 2667)]          0         
_________________________________________________________________
GRU (GRU)                    [(128, 64), (128, 64)]    524736    
_________________________________________________________________
dropout_3 (Dropout)          (128, 64)                 0         
_________________________________________________________________
dense_3 (Dense)              (128, 2667)               173355    
Total params: 698,091
Trainable params: 698,091
Non-trainable params: 0
_________________________________________________________________


Evaluation:  94%|█████████▍| 16/17 [00:07<00:00,  2.08it/s]


	 - Recall@20 epoch 5: 0.300781
	 - MRR@20    epoch 5: 0.104657


Evaluation:  94%|█████████▍| 16/17 [00:07<00:00,  2.13it/s]


	 - Recall@20 epoch 10: 0.321777
	 - MRR@20    epoch 10: 0.111437


Evaluation:  89%|████████▉ | 8/9 [00:03<00:00,  2.05it/s]


====Evaluation====
	 - Recall@20: 0.255859
	 - MRR@20: 0.095412



<IPython.core.display.Javascript object>

세션 설정 단위를 15분으로 설정한 경우, 다른 단위와 비교하여 성능 향상이 이루어지지 않았습니다. 가장 좋은 성능을 보였던 30분으로 세션 단위를 설정합니다.

In [44]:
unit = 0.5
data = get_data(data_orig)

tr, test = split_by_date(data, n_days=200)
tr, val = split_by_date(tr, n_days=200)

id2idx = {item_id: index for index, item_id in enumerate(tr["ItemId"].unique())}

tr = indexing(tr, id2idx)
val = indexing(val, id2idx)
test = indexing(test, id2idx)

<IPython.core.display.Javascript object>

### 배치 사이즈 변경

최초 사용한 모델의 배치 사이즈 128의 절반인 64, 2배인 256으로 실험을 진행합니다.

In [45]:
args = Args(
    tr, val, test, batch_size=64, hsz=64, drop_rate=0.1, lr=0.001, epochs=EPOCH, k=20
)
model = create_model(args)
train_model(model, args, verbose=1)
test_model(model, args, test)

Model: "m_b64_h64"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(64, 1, 2667)]           0         
_________________________________________________________________
GRU (GRU)                    [(64, 64), (64, 64)]      524736    
_________________________________________________________________
dropout_4 (Dropout)          (64, 64)                  0         
_________________________________________________________________
dense_4 (Dense)              (64, 2667)                173355    
Total params: 698,091
Trainable params: 698,091
Non-trainable params: 0
_________________________________________________________________


Evaluation:  97%|█████████▋| 34/35 [00:08<00:00,  4.02it/s]


	 - Recall@20 epoch 5: 0.324449
	 - MRR@20    epoch 5: 0.110769


Evaluation:  97%|█████████▋| 34/35 [00:08<00:00,  4.01it/s]


	 - Recall@20 epoch 10: 0.331342
	 - MRR@20    epoch 10: 0.114148


Evaluation:  95%|█████████▍| 18/19 [00:04<00:00,  3.89it/s]


====Evaluation====
	 - Recall@20: 0.283854
	 - MRR@20: 0.101895



<IPython.core.display.Javascript object>

In [46]:
args = Args(
    tr, val, test, batch_size=256, hsz=64, drop_rate=0.1, lr=0.001, epochs=EPOCH, k=20
)
model = create_model(args)
train_model(model, args, verbose=1)
test_model(model, args, test)

Model: "m_b256_h64"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(256, 1, 2667)]          0         
_________________________________________________________________
GRU (GRU)                    [(256, 64), (256, 64)]    524736    
_________________________________________________________________
dropout_5 (Dropout)          (256, 64)                 0         
_________________________________________________________________
dense_5 (Dense)              (256, 2667)               173355    
Total params: 698,091
Trainable params: 698,091
Non-trainable params: 0
_________________________________________________________________


Evaluation:  88%|████████▊ | 7/8 [00:06<00:00,  1.07it/s]


	 - Recall@20 epoch 5: 0.265625
	 - MRR@20    epoch 5: 0.089999


Evaluation:  88%|████████▊ | 7/8 [00:06<00:00,  1.09it/s]


	 - Recall@20 epoch 10: 0.299665
	 - MRR@20    epoch 10: 0.101935


Evaluation:  75%|███████▌  | 3/4 [00:02<00:00,  1.03it/s]


====Evaluation====
	 - Recall@20: 0.205729
	 - MRR@20: 0.074695



<IPython.core.display.Javascript object>

조금 더 좋은 성능을 보인 64로 설정하여 이후 실험을 진행합니다.

In [47]:
BATCH_SZ = 64

<IPython.core.display.Javascript object>

### Hidden Unit 변경

최초 사용한 모델의 hidden unit size 64의 절반인 32, 2배인 128로 설정하여 실험을 진행합니다.

In [48]:
args = Args(
    tr,
    val,
    test,
    batch_size=BATCH_SZ,
    hsz=32,
    drop_rate=0.1,
    lr=0.001,
    epochs=EPOCH,
    k=20,
)
model = create_model(args)
train_model(model, args, verbose=1)
test_model(model, args, test)

Model: "m_b64_h32"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(64, 1, 2667)]           0         
_________________________________________________________________
GRU (GRU)                    [(64, 32), (64, 32)]      259296    
_________________________________________________________________
dropout_6 (Dropout)          (64, 32)                  0         
_________________________________________________________________
dense_6 (Dense)              (64, 2667)                88011     
Total params: 347,307
Trainable params: 347,307
Non-trainable params: 0
_________________________________________________________________


Evaluation:  97%|█████████▋| 34/35 [00:08<00:00,  4.02it/s]


	 - Recall@20 epoch 5: 0.316176
	 - MRR@20    epoch 5: 0.100552


Evaluation:  97%|█████████▋| 34/35 [00:08<00:00,  4.05it/s]


	 - Recall@20 epoch 10: 0.335018
	 - MRR@20    epoch 10: 0.110685


Evaluation:  95%|█████████▍| 18/19 [00:04<00:00,  3.87it/s]


====Evaluation====
	 - Recall@20: 0.289062
	 - MRR@20: 0.100780



<IPython.core.display.Javascript object>

In [49]:
args = Args(
    tr,
    val,
    test,
    batch_size=BATCH_SZ,
    hsz=128,
    drop_rate=0.1,
    lr=0.001,
    epochs=EPOCH,
    k=20,
)
model = create_model(args)
train_model(model, args, verbose=1)
test_model(model, args, test)

Model: "m_b64_h128"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(64, 1, 2667)]           0         
_________________________________________________________________
GRU (GRU)                    [(64, 128), (64, 128)]    1074048   
_________________________________________________________________
dropout_7 (Dropout)          (64, 128)                 0         
_________________________________________________________________
dense_7 (Dense)              (64, 2667)                344043    
Total params: 1,418,091
Trainable params: 1,418,091
Non-trainable params: 0
_________________________________________________________________


Evaluation:  97%|█████████▋| 34/35 [00:08<00:00,  4.00it/s]


	 - Recall@20 epoch 5: 0.326287
	 - MRR@20    epoch 5: 0.109023


Evaluation:  97%|█████████▋| 34/35 [00:08<00:00,  3.97it/s]


	 - Recall@20 epoch 10: 0.323989
	 - MRR@20    epoch 10: 0.103040


Evaluation:  95%|█████████▍| 18/19 [00:04<00:00,  3.89it/s]


====Evaluation====
	 - Recall@20: 0.269965
	 - MRR@20: 0.097871



<IPython.core.display.Javascript object>

## 실험결과
| Try | Session Size | Batch Size | Hidden Unit Size | Recall | MRR |
| --- | --- | --- | --- | --- | --- |
| #1 | 1 hour | 128 | 64 | 0.170 | 0.055 |
| #2 | 30 min | 128 | 64 | 0.264 | 0.099 |
| #3 | 2 hour | 128 | 64 | 0.266 | 0.092 |
| #4 | 15 min | 128 | 64 | 0.256 | 0.095 |
| #5 | 30 min | 64 | 64 | <ins>0.284</ins> | **0.102** |
| #6 | 30 min | 256 | 64 | 0.206 | 0.075 |
| #7 | 30 min | 64 | 32 | **0.289** | <ins>0.101</ins> |
| #8 | 30 min | 64 | 128 | 0.270 | 0.098 |


## 결론

- 동일한 구조의 네트워크를 사용하여도 세션의 정의에 따라 데이터셋이 달라져 다른 성능을 보였습니다.
    - 세션을 정의하는 시간의 범위가 넓어지거나 좁아짐에 따른 성능의 변화는 일관되지 않아, 목적에 맞는 적절한 세션의 범위를 찾기 위한 과정이 필요합니다.
- 배치 사이즈에 따라 성능이 달라짐을 확인하였습니다.
    - 기준이 되는 128보다 작은 64에서는 성능 향상, 128보다 큰 256에서는 성능 하락을 확인하였습니다.
- 네트워크를 구성하는 GRU 레이어의 hidden unit 사이즈의 크기가 달라짐에 따라 성능이 변화하였습니다.    
    - 기준이 되는 64보다 큰 128에서는 성능이 하락되었고, 64보다 작은 32에서는 성능이 향상되었습니다. 데이터셋과 네트워크에 적절한 탐색이 필요합니다.
- 세션을 정의하는 시간의 범위를 30분, 배치 사이즈를 64, hidden unit 사이즈를 32로 설정하였을 때, 가장 높은 recall을 얻을 수 있었습니다.

## 루브릭

| 평가문항	| 상세기준 |
| --- | --- |
| 1. Movielens 데이터셋을 session based recommendation 관점으로 전처리하는 과정이 체계적으로 진행되었다. | 데이터셋의 면밀한 분석을 토대로 세션단위 정의 과정(길이분석, 시간분석)을 합리적으로 수행한 과정이 기술되었다. |
| 2. RNN 기반의 예측 모델이 정상적으로 구성되어 안정적으로 훈련이 진행되었다. | 적절한 epoch만큼의 학습이 진행되는 과정에서 train loss가 안정적으로 감소하고, validation 단계에서의 Recall, MRR이 개선되는 것이 확인된다. |
| 3. 세션정의, 모델구조, 하이퍼파라미터 등을 변경해서 실험하여 Recall, MRR 등의 변화추이를 관찰하였다. | 3가지 이상의 변화를 시도하고 그 실험결과를 체계적으로 분석하였다. |

### 루브릭 자체평가
1. 세션의 단위를 정의하고, 실험을 통해 서로 다른 정의에 따른 모델의 성능 변화를 확인하였다.
2. 모델이 학습을 진행함에 따라 train loss가 감소하고, recall과 mrr이 향상됨을 확인할 수 있었다.
3. 세션 단위 변경, 모델 변경, 배치 사이즈와 같은 학습 관련 하이퍼파라미터 변경 등의 다양한 실험을 진행하였다.